In [ ]:
import pandas as pd
import os
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Load both Excel files, treating all columns as strings (important for preserving data integrity)
df_compared_to = pd.read_excel(r"data\Anna_sent\Kunden wie Medwing 02.06.25.xlsx", dtype=str)
df_getting_changed = pd.read_excel(r"single_output\Adressen Blister-schoko-Kanabis_27.05.25_deduped.xlsx", dtype=str)

# Optional: Drop internal duplicates within each file by URL
df_getting_changed = df_getting_changed.drop_duplicates(subset=["URL"])
df_compared_to = df_compared_to.drop_duplicates(subset=["URL"])

# Ensure comparison_logs folder exists
os.makedirs("comparison_logs", exist_ok=True)

# Ensure comparison_logs folder exists
os.makedirs("comparison_output", exist_ok=True)

# Step 1: Identify rows in A that have matching URLs in B
matching_urls = df_getting_changed[df_getting_changed["URL"].isin(df_compared_to["URL"])]

# Step 2: Remove those rows from A
df_result = df_getting_changed[~df_getting_changed["URL"].isin(df_compared_to["URL"])]

# Step 3: Save outputs
matching_urls.to_excel(f"comparison_logs/matching_urls_{timestamp}.xlsx", index=False)

# Save cleaned result with formatting
from openpyxl.utils import get_column_letter

with pd.ExcelWriter(f"comparison_output\cleaned_data_{timestamp}.xlsx", engine="openpyxl") as writer:
    df_result.to_excel(writer, index=False, sheet_name="Sheet1")

    worksheet = writer.sheets["Sheet1"]

    # 1. Format 'Number' column as text
    if "Number" in df_result.columns:
        number_col_idx = df_result.columns.get_loc("Number")
        number_col_letter = get_column_letter(number_col_idx + 1)  # openpyxl columns are 1-based

        for row in range(2, len(df_result) + 2):  # Skip header, Excel rows are 1-based
            cell = worksheet[f"{number_col_letter}{row}"]
            cell.number_format = "@"

    # 2. Auto-adjust column widths
    for col_idx, column_cells in enumerate(worksheet.columns, 1):
        max_length = max(
            len(str(cell.value)) if cell.value is not None else 0
            for cell in column_cells
        )
        adjusted_width = max_length + 2
        col_letter = get_column_letter(col_idx)
        worksheet.column_dimensions[col_letter].width = adjusted_width

print(f"✅ Final cleaned data saved to cleaned_data_{timestamp}.xlsx")
print(f"Original A rows: {len(df_getting_changed)}")
print(f"Removed due to URL match: {len(matching_urls)}")
print(f"Final cleaned A rows: {len(df_result)}")

✅ Final cleaned data saved to cleaned_data_20250526_121351.xlsx
Original A rows: 6615
Removed due to URL match: 2465
Final cleaned A rows: 4150
